# Covid-19 Cases in Canada
> "This page tracks the coronavirus epidemic in Canada"
- toc: false
- branch: master
- badges: true
- comments: false
- categories: [charts,visualization,data]

![](../images/canada_covid.png)

In [2]:
#hide
import pandas as pd
import ipywidgets as widgets
from datetime import datetime

CSSE_URL = '../data/covid'
CONFIRMED_URL = f'{CSSE_URL}/canada-confirmed.csv'
DEATHS_URL = f'{CSSE_URL}/canada-deaths.csv'
RECOVERED_URL = f'{CSSE_URL}/canada-recovered.csv'

confirmed_wide = pd.read_csv(CONFIRMED_URL)
deaths_wide = pd.read_csv(DEATHS_URL)
recovered_wide = pd.read_csv(RECOVERED_URL)

In [3]:
#hide
dates = confirmed_wide.columns[4:]

ID_VARS = ['Province/State', 'Country/Region', 'Lat', 'Long']
CASE_COLS = ['Confirmed', 'Active', 'Deaths', 'Recovered']

# Convert from the wide format to long format
confirmed_long = confirmed_wide.melt(id_vars=ID_VARS, value_vars=dates, var_name='Date', value_name='Confirmed')
deaths_long = deaths_wide.melt(id_vars=ID_VARS, value_vars=dates, var_name='Date', value_name='Deaths')
recovered_long = recovered_wide.melt(id_vars=ID_VARS, value_vars=dates, var_name='Date', value_name='Recovered')
covid_full = pd.concat([confirmed_long, deaths_long['Deaths'], recovered_long['Recovered']], axis=1, sort=False)

covid_full.Date = pd.to_datetime(covid_full.Date).dt.normalize()
covid_full = covid_full[['Date', 'Province/State','Lat','Long','Confirmed','Deaths','Recovered']]
covid_full = covid_full.rename(columns={'Province/State': 'Province'})
covid_full = covid_full.set_index('Date')

# Add an Active columns
covid_full['Active'] = covid_full['Confirmed'] - covid_full['Deaths'] - covid_full['Recovered']

covid_full.tail()

provinces = covid_full['Province'].drop_duplicates().to_list()
latest_date = datetime.strftime(covid_full.index.max(), '%A %B %d, %Y')

In [4]:
#collapse-hide
canada_cases = covid_full.groupby(['Date','Province'])[CASE_COLS] \
            .sum().reset_index().melt(id_vars=['Province', 'Date'], 
                                      value_vars=['Recovered', 'Deaths', 'Active'],
                                      var_name='Case', value_name='Count')
canada_cases.Count = canada_cases.Count.astype(int)

In [8]:
#collapse-hide
import altair as alt
from altair.expr import datum

base = alt.Chart(canada_cases).mark_area().encode(
            x='Date:T',
            y='Count:Q',
            color='Case:N'
        ).properties(
            title='title',
            height=120,
            width=160
            )

chart = alt.hconcat()
for province in ['Ontario', 'British Columbia', 'Alberta', 'Quebec']:
    chart |= base.transform_filter(
                datum.Province == province
            ).properties(title=f'{province}')

## Provincial Cases March 16th 2020

In [7]:
#hide_input
chart

alt.HConcatChart(...)